## Import required packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import holoviews as hv
import hvplot.pandas
import panel as pn
import data_processing_methods as dpm
from sklearn.gaussian_process import GaussianProcessRegressor
import plotly.graph_objects as go
import plotly.express as px
pn.extension('plotly')
import datetime as dt

## dashboard for predicting O2 output

In [ ]:
# create widgets
text_box_1 = pn.widgets.TextInput(name='Temperature:', value='1100')
text_box_2 = pn.widgets.TextInput(name='Burner apporx. value:', value='0')
text_box_3 = pn.widgets.TextInput(name='Fan set point:', value='78')
text_box_4 = pn.widgets.TextInput(name='Damper range (high):', value='50')
text_box_5 = pn.widgets.TextInput(name='Damper range (low)', value='96')

# Read the dataframe
burner_settings_df = pd.read_excel('Burners_settings_vs_O2 _rev1.xlsx', sheet_name='Sheet1')
standardised_df = burner_settings_df.copy()
for col in burner_settings_df.columns:

    if (np.std(burner_settings_df[col])> 0):
        standardised_df[col] = dpm.standardise(burner_settings_df[col],np.mean(burner_settings_df[col]),np.std(burner_settings_df[col]))
    else:
        standardised_df = standardised_df.drop(columns=col)

X = standardised_df.drop(columns =['Output O2 / %', 'Output Burner usage / %'])
y = standardised_df['Output O2 / %']

# Drop columns with corr coeffcients higher than 0.9 
X = X.drop(columns =['Burner turns from zero', 'Fan speed / RPM'])
X_train = X[10:]
y_train = y[10:]

# Train Gaussian Process regression model
gpr = GaussianProcessRegressor().fit(X_train, y_train)
y_std = np.std(burner_settings_df['Output O2 / %'])
y_mean = np.mean(burner_settings_df['Output O2 / %'])

# define the plot functions for predicting O2 values
def plot_predictions():

    X_predict = X
    y_predict = gpr.predict(X_predict)
    y_actual = y
    Y_df = pd.DataFrame(y_predict*y_std+y_mean, columns=['Predicted'])
    Y_df['Actual'] = y_actual*y_std+y_mean 
    
    fig_a = px.line(Y_df, labels={
                     "value": "O2 output",
                 }, title='O2 predictions',height=600, width=920, markers=True)

    return fig_a

# define the plot functions for new user defined inputs
def plot_predictions_b(text_box_1,text_box_2,text_box_3,text_box_4,text_box_5):

    cols = list(X_train.columns)
    X_new = pd.DataFrame([[int(text_box_1),int(text_box_2),int(text_box_3),int(text_box_4),int(text_box_5)]],
                         columns = cols)
    X_new_pr= X_new.copy()
    for col in cols:    
        X_new_pr[col] = dpm.standardise(X_new[col],np.mean(burner_settings_df[col]),np.std(burner_settings_df[col]))

    y_new_predict = gpr.predict(X_new_pr)
    Y_new = pd.DataFrame(y_new_predict*y_std+y_mean, columns=['Newly Predicted'])
    
    fig_b = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = int(y_new_predict*y_std+y_mean),
    title = {'text': "02 output"},
    domain = {'x': [0, 1], 'y': [0, 1]}))
    
    return fig_b


In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Output O2 predictions
""", width=500)

# Create interactive panels
layout_b_gague = pn.interact(plot_predictions_b, text_box_1 = text_box_1,text_box_2 = text_box_2,
                             text_box_3 = text_box_3,text_box_4 = text_box_4,text_box_5 = text_box_5)
layout_b_plot = pn.Row(plot_predictions,layout_b_gague)
layout_b = pn.Column(explanation_pane,layout_b_plot)
layout_b.servable()

# Main effects plots

In [ ]:
# No. samples to be generated
N_samples = 1000
cols = list(X_train.columns)

# Initialise arrays to store samples of un-standardised and 
# standardised inputs
D = len (cols)
X_samples_us = np.zeros([N_samples, D])
X_samples = np.zeros([N_samples, D])

for i in range(N_samples):
    for j in range (D):
        X_samples_us[i, j] = np.random.uniform(np.min(burner_settings_df[cols[j]]), np.max(burner_settings_df[cols[j]]))
        
# Standardise the samples created
for j in range (D):    
    X_samples[:,j] = dpm.standardise(X_samples_us[:,j],np.mean(burner_settings_df[cols[j]]),np.std(burner_settings_df[cols[j]]))

# Save the predictions 
X_samples_df = pd.DataFrame(data = X_samples, columns = cols)
X_samples_us_df = pd.DataFrame(data = X_samples_us, columns = cols)
y_samples_predict = gpr.predict(X_samples_df)
y_samples_predict = y_samples_predict*y_std+y_mean
X_samples_us_df['Predicted O2 %'] = y_samples_predict

cols_samples = list(X_samples_us_df.columns)
y_samples = pn.widgets.Select(name='Chose a signal', options=cols_samples ,value='Temperature setpoint / degC')

@pn.depends(y_samples.param.value)
def plot_main_effects(y_samples):
    # Find the mean of each bin (binning data)
    n_bins = 30
    x_plot = X_samples_us_df[y_samples]
    y_plot = X_samples_us_df['Predicted O2 %']

    bins = np.linspace(np.min(x_plot), np.max(x_plot), n_bins)
    main_effect = np.zeros(len(bins)-1)
    main_effect_index = np.zeros(len(bins)-1)
    main_effect_df = pd.DataFrame({})

    
    for j in range(len(bins)-1):
        indx = np.logical_and(x_plot > bins[j], x_plot < bins[j+1])
        main_effect_index[j] = 0.5*(bins[j] + bins[j+1])

        # Only compute mean if there are any points in bin
        if np.sum(indx) > 0:
            main_effect[j] = np.mean(y_plot[indx])

    main_effect_df['index'] = main_effect_index
    main_effect_df['value'] = main_effect
    
    # plot main effects of model inputs
    fig_a = X_samples_us_df.hvplot.scatter(x = y_samples, y = 'Predicted O2 %', height = 500, width = 920, hover_cols = 'all')
    fig_b = main_effect_df.hvplot.line(x = 'index', y = 'value', xlim =[np.min(x_plot), np.max(x_plot)],color= 'red')
    fig = fig_a*fig_b
    return fig

In [ ]:
# Plot  
explanation_pane = pn.pane.Markdown("""
# Main effects
""", width=500)

# Create interactive panels
chart_interact = pn.interact(plot_main_effects, y_samples = y_samples)
layout = pn.Column(explanation_pane, chart_interact)
layout.servable()

## dashboard for further visualising

In [ ]:
# create widgets
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

cols_1 = list(df_merged.columns[df_merged.columns.str.startswith('ROOF_0104_300_')])
y1 = pn.widgets.Select(name='Chose a signal to tune filtering parameters', options=cols_1 ,value='ROOF_0104_300_10_TC')

slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting sensor data
@pn.depends(y1.param.value)
def plot_trunc_furnace_data(date_start, date_end, y1,slider_pos,Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('ROOF_0104_300_')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_a = df_merged_truncated.hvplot(height = 400, width = 920, line_width = 3,grid= True,legend='left')
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,y1] = dpm.remove_spikes(df_merged_truncated.loc[:,y1],olr_def=1)
    
    df_merged_filtered.loc[:,y1] = dpm.low_pass_filter(df_merged_filtered.loc[:,y1],wn=slider_pos)
    fig_b = df_merged_filtered.hvplot.line(y = y1, height = 400, width = 920, color= 'black', label = 'Filtered signal',line_width = 2)
    return fig_a*fig_b

In [ ]:
# Plot  
explanation_pane = pn.pane.Markdown("""
# Roof temperatures
""", width=500)

# Create interactive panels
chart_b_interact = pn.interact(plot_trunc_furnace_data,date_start=dt_start_input, date_end=dt_end_input,y1=y1,slider_pos=slider)
layout_b = pn.Column(explanation_pane, chart_b_interact)

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

# create widgets
cols_2 = list(df_merged.columns[df_merged.columns.str.startswith('PID_ZONE_1')])
y2 = pn.widgets.Select(name='Chose a signal to tune filter parameters', options=cols_2 ,value='PID_ZONE_1_PV')

slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting PID zone 1 sensor data
@pn.depends(y2.param.value)
def plot_trunc_PID_data(date_start, date_end, y2,slider_pos,Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('PID_ZONE_1')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_a = df_merged_truncated.hvplot(height = 400, width = 920, line_width = 3,grid= True,legend='left')
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,y2] = dpm.remove_spikes(df_merged_truncated.loc[:,y2],olr_def=1)
    
    df_merged_filtered.loc[:,y2] = dpm.low_pass_filter(df_merged_filtered.loc[:,y2],wn=slider_pos)
    fig_b = df_merged_filtered.hvplot.line(y = y2, height = 400, width = 920, color= 'black', label = 'Filtered signal',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# PID Zone 1
""", width=500)

# Create interactive panels
chart_c_interact = pn.interact(plot_trunc_PID_data,date_start=dt_start_input, date_end=dt_end_input,y2=y2,slider_pos=slider)
layout_c = pn.Column(explanation_pane, chart_c_interact)

layout = pn.Row(layout_b,layout_c)
layout.servable()

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

# create widgets
slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting  O2 sensor data
def plot_O2_data(slider_pos, date_start, date_end, Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('FURNACE_0126_341_04_O2')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_a = df_merged_truncated.hvplot(ylabel = 'O2 output',height = 500, width = 1600, line_width = 5, grid= True,legend='left')
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'] = dpm.remove_spikes(df_merged_truncated.loc[:,'FURNACE_0126_341_04_O2'],olr_def=1)
    
    df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'] = dpm.low_pass_filter(df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'],wn=slider_pos)
    fig_b = df_merged_filtered.hvplot.line(y = 'FURNACE_0126_341_04_O2', height = 500, width = 1600, color= 'black', label = 'Filtered signal',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Output O2
""", width=500)

# Create interactive panels
chart_b_interact = pn.interact(plot_O2_data,date_start=dt_start_input, date_end=dt_end_input,slider_pos=slider)
layout_b = pn.Column(explanation_pane, chart_b_interact)
layout_b.servable()